In [1]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


In [3]:
from bs4 import BeautifulSoup 

In [4]:
import html5lib

In [5]:
import lxml

## Scrape data set from Wikipedia website and organize it into a pandas DataFrame

In [6]:
# read the webpage from the internet, then scrape the text data set
sourceHTML = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(sourceHTML.content, 'lxml')

In [7]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)

In [8]:
df=pd.DataFrame(table_contents)

In [9]:
df.shape

(103, 3)

In [10]:
df.head()

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Queen's Park,Ontario Provincial Government,M7A


In [13]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [11]:
df.head()

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Queen's Park,Ontario Provincial Government,M7A


## Read in geo coordinate data from a given resource to create a new dataframe then combine with the previous dataframe

In [12]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')      # read in the geo-location data set related to all of the postal codes in city Toronto

In [13]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_geo.shape

(103, 3)

In [15]:
df_geo = df_geo.rename(columns = {'Postal Code':'PostalCode'})       # adjust the column name to what we need

In [16]:
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df.head()

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Queen's Park,Ontario Provincial Government,M7A


In [18]:
DF_geo = pd.merge(df, df_geo, on = 'PostalCode')

In [19]:
DF_geo.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [20]:
DF_geo.shape

(103, 5)

## Create a neighborhood map for city Toronto

In [21]:
address = 'Toronto, Ca'                         

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)                   
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [22]:
# first, creat city map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# then add markers of neighborhood information to the city map
for lat, lng, borough, neighborhood in zip(DF_geo['Latitude'], DF_geo['Longitude'], DF_geo['Borough'], DF_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

In [23]:
map_toronto

## Explore the neighborhoods of city Toronto

In [24]:
DF_geo.Borough.unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East YorkEast Toronto', 'Central Toronto',
       'MississaugaCanada Post Gateway Processing Centre',
       'Downtown TorontoStn A PO Boxes25 The Esplanade',
       'EtobicokeNorthwest',
       'East TorontoBusiness reply mail Processing Centre969 Eastern'],
      dtype=object)

In [25]:
# get data set of 'Downtown Toronto' borough
Downtown_data = DF_geo[DF_geo['Borough'] == 'Downtown Toronto'].reset_index(drop=True) # to extract all relevant data in one specific borough into to a new dataframe
Downtown_data.shape

(17, 5)

In [26]:
# get data set in 'Central Toronto' borough
Central_data = DF_geo[DF_geo['Borough'] == 'Central Toronto'] .reset_index(drop=True) # to extract all relevant data in one specific borough into to a new dataframe
Central_data.shape

(9, 5)

In [27]:
# get data set in 'East Toronto' borough
East_data = DF_geo[DF_geo['Borough'] == 'East Toronto'].reset_index(drop=True) # to extract all relevant data in one specific borough into to a new dataframe
East_data.shape

(4, 5)

In [28]:
# get data set in 'West Toronto' borough
West_data = DF_geo[DF_geo['Borough'] == 'West Toronto'].reset_index(drop=True) # to extract all relevant data in one specific borough into to a new dataframe
West_data.shape

(6, 5)

In [29]:
torontos = Downtown_data.append(Central_data)
torontos = torontos.append(East_data)
torontos = torontos.append(West_data)
torontos = torontos.reset_index(drop = True)                                        # reset the index after the merges
torontos = torontos.rename(columns = {'Neighbourhood':'Neighborhood'})   # adjust the name a column for convenience of later operations
#torontos.head()
torontos.shape

(36, 5)

In [30]:
address = 'Downtown, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [31]:
torontos.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
1,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
2,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
3,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
4,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383


#### Visualize the new dataframe

In [32]:
map_torontos = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map representing the neighborhood of new Toronto data set
for lat, lng, label in zip(torontos['Latitude'], torontos['Longitude'], torontos['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torontos)  
    
map_torontos

## Utilizing Foursquare API to explore venues around just one neighborhood as a try out

In [66]:
CLIENT_ID = 'ABCD' # your Foursquare ID
CLIENT_SECRET = 'PQRS' # your Foursquare Secret
ACCESS_TOKEN = 'XYZ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ABCD
CLIENT_SECRET:PQRS


In [34]:
torontos.iloc[0,0]

'Downtown Toronto'

In [35]:
neighborhood_latitude = torontos.iloc[0, 3] # neighborhood latitude value
neighborhood_longitude = torontos.iloc[0, 4] # neighborhood longitude value

neighborhood_name = torontos.iloc[0, 0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Downtown Toronto are 43.6542599, -79.3606359.


#### Get the top 100 venues in Rosedale within a radius of 1000 meters

In [36]:
radius = 1000     # considering the less population density distribution of canada, using a larger radius 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius,
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=31TGRMWWKUGLYRAJ2I40YSOEKHLSB1Z43U3GXVNK2MT1ZLPQ&client_secret=33FCH2XYAAT5ONXPJNWTX0B1KAUWOPUENRAGAG52OICO5SQN&v=20180604&ll=43.6542599,-79.3606359&radius=1000&limit=100'

In [37]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606bf701c27ed34e04f79f72'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 111,
  'suggestedBounds': {'ne': {'lat': 43.66325990900001,
    'lng': -79.3482199002972},
   'sw': {'lat': 43.64525989099999, 'lng': -79.37305189970282}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65355870959944,
          'lng': -79.36180945913513}],
        'distance':

In [38]:
results.keys()

dict_keys(['meta', 'response'])

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON in a pd df

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]           # keep the columns we need

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)    # apply the function from the above

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]      # normalize the column name structure and only keep the last portion of original name of the columns

In [41]:
nearby_venues
print('There are {} venues around "Rosedale" within 1000 meters.'.format(nearby_venues.shape[0]))

There are 100 venues around "Rosedale" within 1000 meters.


In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):  
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']    
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results]) 

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list]) 
    nearby_venues.columns = ['Neighborhood',                                                
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',  
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:

Toronto_venues = getNearbyVenues(names=torontos['Neighborhood'],
                                   latitudes=torontos['Latitude'],
                                   longitudes=torontos['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles
Runny

In [44]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2934, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [45]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100
Davisville North,100,100,100,100,100,100


In [46]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


## Analyze Each Neighborhood

In [47]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")   # transform the values under'Venuess Category' into attribute variables

# add neighborhood column back to dataframe   
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']                                 # restructure the dataframe with a new column

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])                # a new list of column name
Toronto_onehot = Toronto_onehot[fixed_columns]                                                  # a new dataframe with columns adjusted

Toronto_onehot.head()


,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [48]:
Toronto_onehot.shape

(2934, 269)

#### Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [49]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.00,0.010000,0.00,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.01,0.010000,

In [50]:
Toronto_grouped.shape

(36, 269)

In [51]:
num_top_venues = 5
count = 0
for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()        # transpose the row of most hot top column names into a columne called 'venue'
    temp.columns = ['venue','freq']                                                        # construct a new column 'freq' containing the values of the respective values of 'venue'
    temp = temp.iloc[1:]                                                                   # index start from the second row on
    temp['freq'] = temp['freq'].astype(float)                                              # define the data type
    temp = temp.round({'freq': 2})                                                         # regulate the round up of 'freq'
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))    # in a decending order
    print('\n')
    count +=1
print('count the total:',count)

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.07
2                Hotel  0.05
3  Japanese Restaurant  0.04
4                 Park  0.04


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.07
1   Restaurant  0.06
2          Bar  0.06
3  Coffee Shop  0.05
4       Bakery  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0  Harbor / Marina  0.13
1             Café  0.13
2      Coffee Shop  0.13
3           Garden  0.07
4   Scenic Lookout  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.04
2  Japanese Restaurant  0.03
3      Bubble Tea Shop  0.03
4                 Park  0.03


----Christie----
               venue  freq
0  Korean Restaurant  0.12
1        Coffee Shop  0.07
2               Café  0.07
3      Grocery Store  0.06
4

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):                                             # develop the column variables
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)    # call return data from function and sign to new dataframe structure

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Beer Bar,Restaurant,Creperie,Gym,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Restaurant,Coffee Shop,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Lounge,Thrift / Vintage Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Harbor / Marina,Coffee Shop,Café,Scenic Lookout,Dance Studio,Airport,Airport Lounge,Dog Run,Sculpture Garden,Garden
3,Central Bay Street,Coffee Shop,Café,Park,Bubble Tea Shop,Japanese Restaurant,Ramen Restaurant,Pizza Place,Mexican Restaurant,Juice Bar,Theater
4,Christie,Korean Restaurant,Coffee Shop,Café,Grocery Store,Pizza Place,Cocktail Bar,Mexican Restaurant,Ethiopian Restaurant,Park,Sandwich Place


In [54]:
neighborhoods_venues_sorted.shape

(36, 11)

## Cluster the Neighborhoods

In [55]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)   

# check cluster labels generated for each row in the dataframe
kmeans.labels_.size, kmeans.labels_

(36, array([0, 3, 4, 0, 3, 0, 0, 2, 2, 2, 0, 2, 0, 0, 3, 3, 3, 1, 3, 2, 2, 3,
        0, 0, 0, 2, 2, 0, 0, 3, 2, 3, 3, 3, 0, 3]))

In [56]:
torontos.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
1,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
2,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
3,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
4,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383


In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) # at position 0 insert a new column applying the values of kmeans.labels_

Toronto_merged = torontos

# merge Toronto_grouped with torontos to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,0,Coffee Shop,Restaurant,Pub,Park,Theater,Café,Italian Restaurant,Diner,Breakfast Spot,Sushi Restaurant
1,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,0,Coffee Shop,Gastropub,Japanese Restaurant,Ramen Restaurant,Bookstore,Cosmetics Shop,Theater,Café,Restaurant,Department Store
2,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Gastropub,Japanese Restaurant,Seafood Restaurant,Cosmetics Shop,Italian Restaurant,Creperie,Bakery
3,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Beer Bar,Restaurant,Creperie,Gym,Seafood Restaurant
4,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383,0,Coffee Shop,Café,Park,Bubble Tea Shop,Japanese Restaurant,Ramen Restaurant,Pizza Place,Mexican Restaurant,Juice Bar,Theater


In [58]:
Toronto_merged.shape

(36, 16)

### Let's visualize the resulting clusters

In [59]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [60]:
map_clusters

## Examine Clusters

In [61]:

Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Restaurant,Pub,Park,Theater,Café,Italian Restaurant,Diner,Breakfast Spot,Sushi Restaurant
1,Downtown Toronto,0,Coffee Shop,Gastropub,Japanese Restaurant,Ramen Restaurant,Bookstore,Cosmetics Shop,Theater,Café,Restaurant,Department Store
2,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gastropub,Japanese Restaurant,Seafood Restaurant,Cosmetics Shop,Italian Restaurant,Creperie,Bakery
3,Downtown Toronto,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Beer Bar,Restaurant,Creperie,Gym,Seafood Restaurant
4,Downtown Toronto,0,Coffee Shop,Café,Park,Bubble Tea Shop,Japanese Restaurant,Ramen Restaurant,Pizza Place,Mexican Restaurant,Juice Bar,Theater
6,Downtown Toronto,0,Coffee Shop,Café,Hotel,Theater,Gastropub,Beer Bar,Japanese Restaurant,Furniture / Home Store,Clothing Store,Concert Hall
7,Downtown Toronto,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Theater,Brewery,Gym,Scenic Lookout,Plaza
8,Downtown Toronto,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Theater,Restaurant,Beer Bar,Cosmetics Shop,American Restaurant,Thai Restaurant
9,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Concert Hall,Theater,Gastropub,Cosmetics Shop,Beer Bar,Japanese Restaurant
13,Downtown Toronto,0,Coffee Shop,Grocery Store,Park,Breakfast Spot,BBQ Joint,Pie Shop,Sandwich Place,Candy Store,Metro Station,Convenience Store


In [62]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,1,Bookstore,Gym / Fitness Center,College Quad,College Gym,Coffee Shop,Park,Café,Trail,Yoga Studio,Dumpling Restaurant


In [63]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Italian Restaurant,Pharmacy,Coffee Shop,Café,Sushi Restaurant,Bank,Spa,Playground,Gym Pool,Pet Store
19,Central Toronto,2,Coffee Shop,Italian Restaurant,Restaurant,Café,Sushi Restaurant,Pizza Place,Gym,Dessert Shop,Pub,Pharmacy
20,Central Toronto,2,Park,Café,Coffee Shop,Italian Restaurant,Bank,Skating Rink,Burger Joint,Liquor Store,Japanese Restaurant,Sushi Restaurant
21,Central Toronto,2,Coffee Shop,Skating Rink,Italian Restaurant,Café,Restaurant,Mexican Restaurant,Diner,Thai Restaurant,Sporting Goods Shop,Park
23,Central Toronto,2,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Pizza Place,Restaurant,Middle Eastern Restaurant,Gym,Dessert Shop,Indian Restaurant
24,Central Toronto,2,Grocery Store,Park,Italian Restaurant,Coffee Shop,Thai Restaurant,Gym,Pizza Place,Sushi Restaurant,Café,Bank
25,Central Toronto,2,Coffee Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Park,Grocery Store,Restaurant,Gym / Fitness Center,Bank,Bagel Shop
30,West Toronto,2,Coffee Shop,Café,Park,Sushi Restaurant,Italian Restaurant,Bar,Bakery,Convenience Store,Pharmacy,Gourmet Shop
35,West Toronto,2,Coffee Shop,Café,Pizza Place,Bakery,Restaurant,Pub,Italian Restaurant,Sushi Restaurant,Falafel Restaurant,Bar


In [64]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,3,Korean Restaurant,Coffee Shop,Café,Grocery Store,Pizza Place,Cocktail Bar,Mexican Restaurant,Ethiopian Restaurant,Park,Sandwich Place
10,Downtown Toronto,3,Café,Bakery,Restaurant,Bar,Vegetarian / Vegan Restaurant,Beer Bar,Mexican Restaurant,Coffee Shop,Bookstore,Italian Restaurant
11,Downtown Toronto,3,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Art Gallery,Mexican Restaurant,Park,Yoga Studio,Arts & Crafts Store,Beer Bar
22,Central Toronto,3,Italian Restaurant,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Gym,Pub,Museum,Bakery,Grocery Store,Restaurant
26,East Toronto,3,Pub,Coffee Shop,Pizza Place,Beach,Breakfast Spot,Japanese Restaurant,Asian Restaurant,Sandwich Place,Burger Joint,Indian Restaurant
27,East Toronto,3,Greek Restaurant,Café,Coffee Shop,Pub,Bank,Italian Restaurant,Fast Food Restaurant,Yoga Studio,Discount Store,Restaurant
28,East Toronto,3,Indian Restaurant,Park,Coffee Shop,Restaurant,Beach,Fast Food Restaurant,Café,Brewery,Bakery,Sandwich Place
29,East Toronto,3,Coffee Shop,Brewery,American Restaurant,Bar,Bakery,Café,Diner,Vietnamese Restaurant,French Restaurant,Italian Restaurant
31,West Toronto,3,Café,Bar,Restaurant,Vegetarian / Vegan Restaurant,Bakery,Italian Restaurant,Pizza Place,Cocktail Bar,Coffee Shop,Asian Restaurant
32,West Toronto,3,Café,Bar,Restaurant,Coffee Shop,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Lounge,Thrift / Vintage Store


In [65]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,4,Harbor / Marina,Coffee Shop,Café,Scenic Lookout,Dance Studio,Airport,Airport Lounge,Dog Run,Sculpture Garden,Garden
